# Generate more complex anomalies

Anomalies generated have now:
- random size (between 2 and 5)
- random shape (either line, square or circle)
- random place in the image

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.random import normal
import tensorflow.keras as keras

from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import mnist

from MNIST_anomalies import get_rm, contour_anomalies, plot_anomalies
from MNIST_anomalies import plot_predictions

In [2]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = np.expand_dims(x_train, -1).astype('float32') / 255.0
x_test = np.expand_dims(x_test, -1).astype('float32') / 255.0
    
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [3]:
vae = tf.keras.models.load_model("UNet_model.pb")

In [35]:
def set_anomaly(img, an_type="l", an_size=3, s=0):
    """
    Set a "draw line" anomaly on the given image
    an_type: "l": line, "c": circle, "s": square
    an_size: size of anomaly
    s: random seed
    """
    np.random.seed(s)
    square_shape = 28

    if img.shape == (square_shape ** 2,):  # flat format
        img = img.reshape((28, 28))
        
    if img.shape == (28, 28, 1):
        img = np.squeeze(img, axis=-1)
        
    modif_img = np.array(img)
        
    x, y = np.random.randint(0, square_shape, size=2)
    
    if an_type == "l":
        for i in range(an_size - 1):
            if x + i < square_shape and y + i + 1 < square_shape:
                modif_img[x + i, y + i] = .99
                modif_img[x + i, y + i + 1] = .5
                modif_img[x + i, y + i - 1] = .5

            if x - i >= 0 and y - i - 1 >= 0 and y - i + 1 < 28:
                modif_img[x - i, y - i] = .99
                modif_img[x - i, y - i + 1] = .5
                modif_img[x - i, y - i - 1] = .5
                
    if an_type == "c":
        for i in range(an_size):
            for j in range(an_size):
                if np.sqrt(i**2 + j**2) <= an_size:
                    if x + i < square_shape and y + j < square_shape:
                        modif_img[x + i, y + j] = 0.99
                    if x - i >= 0 and y + j < square_shape:
                        modif_img[x - i, y + j] = 0.99
                    if x + i < square_shape and y - j >= 0:
                        modif_img[x + i, y - j] = 0.99
                    if x - i >= 0 and y - j >= 0:
                        modif_img[x - i, y - j] = 0.99
                        
    if an_type == "s":
        for i in range(an_size // 2):
            for j in range(an_size // 2):
                    if x + i < square_shape and y + j < square_shape:
                        modif_img[x + i, y + j] = 0.99
                    if x - i >= 0 and y + j < square_shape:
                        modif_img[x - i, y + j] = 0.99
                    if x + i < square_shape and y - j >= 0:
                        modif_img[x + i, y - j] = 0.99
                    if x - i >= 0 and y - j >= 0:
                        modif_img[x - i, y - j] = 0.99

    return modif_img


def predict_anomalies(model, ref, dims=(28, 28, 1)):
    """
    Make model predictions on reference and reference + anomalies
    tensor (bool): True if the model takes as inputs a rank-3 tensor (28, 28, 1)
    """
    anom_types = ["l", "s", "c"]
    anom_sizes = [2, 3, 4, 5]
        
    predictions = model.predict(ref)
    
    l = []
    for i, x in enumerate(x_test):
        np.random.seed(i)
        t, size = np.random.choice(anom_types), np.random.choice(anom_sizes)
        anom = set_anomaly(x, an_type=t, an_size=size, s=i)
        l.append(anom)
    anomalies = np.array(l)    
    
    anomalies_pred = model.predict(np.expand_dims(anomalies, axis=-1))

    if dims == (28, 28, 1):
        predictions = np.squeeze(predictions, axis=-1)
        anomalies_pred = np.squeeze(anomalies_pred, axis=-1)

    return predictions, anomalies, anomalies_pred

In [36]:
predictions, anomalies, anomalies_pred = predict_anomalies(vae, x_test)

print(predictions.shape)
print(x_test.shape)
print(anomalies.shape)
print(anomalies_pred.shape)

if x_test.shape == (x_test.shape[0], 28, 28, 1):
    x_test = np.squeeze(x_test, axis=-1)

np.save("test_set", x_test)
np.save("predictions_on_test_set", predictions)
np.save("anomalies", anomalies)
np.save("predictions_on_anomalies", anomalies_pred)

(10000, 28, 28)
(10000, 28, 28, 1)
(10000, 28, 28)
(10000, 28, 28)
